# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
with open("openai-api.txt", "r") as file:
    api_key = file.read().strip()

In [2]:


from openai import OpenAI
import os

'''from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv(api_key)'''

'from dotenv import load_dotenv, find_dotenv\n_ = load_dotenv(find_dotenv()) # read local .env file\n\nOPENAI_API_KEY  = os.getenv(api_key)'

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He drove for Red Bull Racing and secured his first of four consecutive drivers' titles that season.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [7]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 Championship was won by Lewis Hamilton of Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [9]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [13]:
from openai import OpenAI
import os

client = OpenAI(api_key= api_key)

model = 'gpt-4o-mini'

In [14]:
#Classification

def city_prompt(description):
    return f"""In the following I am going to give you the description of a city.

###TASK###
Read the TRIP description carefully and return a score for how AFFORDABLE the TRIP is, how SAFE the place is is and how much TOURISTIC the city can be. For each of the three characteristics (BUDGET, SAFE, TOURISTIC) return a score from 1 to 10.

###CONTEXT###
We are going to use these scores to greenlight cities. It is very important to ge these scores correctly as my job depends on that. The company usually greenlights touristic cities.

###FORMAT###
Reply only with the three characteristics (BUDGET, SAFE, TOURISTIC) followed by the score for each of them.



I will now give you the cities

CITY: {description}"""

In [15]:
city1 = "Rio de Janeiro, or simply Rio, is the capital of the state of Rio de Janeiro. It is the second-most-populous city in Brazil (after São Paulo) and the sixth-most-populous city in the Americas. Founded in 1565 by the Portuguese, the city was initially the seat of the Captaincy of Rio de Janeiro, a domain of the Portuguese Empire. In 1763, it became the capital of the State of Brazil, a state of the Portuguese Empire. In 1808, when the Portuguese Royal Court moved to Brazil, Rio de Janeiro became the seat of the court of Queen Maria I of Portugal. She subsequently, under the leadership of her son the prince regent John VI of Portugal, raised Brazil to the dignity of a kingdom, within the United Kingdom of Portugal, Brazil, and Algarves. Rio remained as the capital of the pluricontinental monarchy until 1822, when the Brazilian War of Independence began. This is one of the few instances in history that the capital of a colonizing country officially shifted to a city in one of its colonies. Rio de Janeiro subsequently served as the capital of the independent monarchy, the Empire of Brazil, until 1889, and then the capital of a republican Brazil until 1960 when the capital was transferred to Brasília."
city2 = "Bogotá, officially Bogotá, Distrito Capital, abbreviated Bogotá, D.C., and formerly known as Santa Fe de Bogotá (Spanish: [ˌsanta ˈfe ðe βoɣoˈta]; lit. 'Holy Faith of Bogotá') during the Spanish Colonial period and between 1991 and 2000, is the capital and largest city of Colombia, and one of the largest cities in the world. The city is administered as the Capital District, as well as the capital of, though not politically part of, the surrounding department of Cundinamarca.[16] Bogotá is a territorial entity of the first order, with the same administrative status as the departments of Colombia. It is the main political, economic, administrative, industrial, cultural, aeronautical, technological, scientific, medical and educational center of the country and northern South America. Bogotá was founded as the capital of the New Kingdom of Granada on 6 August 1538 by Spanish conquistador Gonzalo Jiménez de Quesada after a harsh expedition into the Andes conquering the Muisca, the indigenous inhabitants of the Altiplano. Santafé (its name after 1540) became the seat of the government of the Spanish Royal Audiencia of the New Kingdom of Granada (created in 1550), and then after 1717 it was the capital of the Viceroyalty of New Granada. After the Battle of Boyacá on 7 August 1819, Bogotá became the capital of the independent nation of Gran Colombia. It was Simón Bolívar who rebaptized the city with the name of Bogotá, as a way of honoring the Muisca people and as an emancipation act towards the Spanish crown.[23] Hence, since the Viceroyalty of New Granada's independence from the Spanish Empire and during the formation of present-day Colombia, Bogotá has remained the capital of this territory."
city3 = "Zurich is the largest city in Switzerland and the capital of the canton of Zurich. It is located in north-central Switzerland,[5] at the northwestern tip of Lake Zurich. As of January 2023, the municipality had 443,037 inhabitants,[6] the urban area 1.315 million (2009),[7] and the Zurich metropolitan area 1.83 million (2011).[8] Zurich is a hub for railways, roads, and air traffic. Both Zurich Airport and Zurich's main railway station are the largest and busiest in the country. Permanently settled for over 2,000 years, Zurich was founded by the Romans, who called it Turicum. However, early settlements have been found dating back more than 6,400 years (although this only indicates human presence in the area and not the presence of a town that early).[9] During the Middle Ages, Zurich gained the independent and privileged status of imperial immediacy and, in 1519, became a primary centre of the Protestant Reformation in Europe under the leadership of Huldrych Zwingli.[10]"
city4 = "Moscow[a] is the capital and largest city of Russia. The city stands on the Moskva River in Central Russia, with a population estimated at over 13 million residents within the city limits,[6] over 19.1 million residents in the urban area,[7] and over 21.5 million residents in its metropolitan area.[14] The city covers an area of 2,511 square kilometers (970 sq mi), while the urban area covers 5,891 square kilometers (2,275 sq mi),[7] and the metropolitan area covers over 26,000 square kilometers (10,000 sq mi).[14] Moscow is among the world's largest cities, being the most populous city in its entirety in Europe, the largest urban and metropolitan area in Europe,[7][14] and the largest city by land area on the European continent.[15] First documented in 1147, Moscow grew to serve as the capital of the Grand Duchy of Moscow. When the Tsardom of Russia was proclaimed, Moscow remained the political and economic center for most of its history. Under the reign of Peter the Great, the Russian capital was moved to the newly founded city of Saint Petersburg in 1712, decreasing Moscow's influence. Following the Russian Revolution and the establishment of the Russian SFSR, the capital was moved back to Moscow in 1918, where it later became the political center of the Soviet Union.[16] In the aftermath of the dissolution of the Soviet Union, Moscow remained the capital city of the newly established Russian Federation."

In [27]:
response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content":"You are a discerning movie critic"},
                {"role": "user", "content": city_prompt(city1)} ],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET: 6  
SAFE: 5  
TOURISTIC: 9  


In [26]:
response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content":"You are a discerning movie critic"},
                {"role": "user", "content": city_prompt(city2)} ],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET: 7  
SAFE: 5  
TOURISTIC: 8  


In [18]:
response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content":"You are a discerning movie critic"},
                {"role": "user", "content": city_prompt(city3)} ],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET: 3  
SAFE: 9  
TOURISTIC: 7  


In [19]:
response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content":"You are a discerning movie critic"},
                {"role": "user", "content": city_prompt(city4)} ],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET: 5  
SAFE: 6  
TOURISTIC: 8  


In [20]:
#Few Shots

messages_with_few_shot=[
    {"role": "system", "content":"You are a expert travel consultant"},
    {"role": "user", "content": """In the following I am going to give you the specs of a TRIP.

###TASK###
Read the TRIP description carefully and return a score for how AFFORDABLE the TRIP is, how SAFE the place is is and how much TOURISTIC the city can be. For each of the three characteristics (BUDGET, SAFE, TOURISTIC) return a score from 1 to 10.

###CONTEXT###
We are going to use these scores to greenlight cities. It is very important to ge these scores correctly as my job depends on that. The company usually greenlights touristic cities.

###FORMAT###
Reply only with the three characteristics (BUDGET, SAFE, TOURISTIC) followed by the score for each of them.


I will now give you the movies

CITY: Paris is the capital and largest city of France. With an estimated population of 2,102,650 residents in January 2023[2] in an area of more than 105 km2 (41 sq mi),[5] Paris is the fourth-most populous city in the European Union, the ninth-most populous city in Europe and the 30th most densely populated city in the world in 2022.[6] Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. Because of its leading role in the arts and sciences and its early adaptation of extensive street lighting, it became known as the City of Light in the 19th century.[7]
The City of Paris is the centre of the Île-de-France region, or Paris Region, with an official estimated population of 12,271,794 inhabitants in January 2023, or about 19% of the population of France.[2] The Paris Region had a nominal GDP of €765 billion (US$1.064 trillion when adjusted for PPP)[8] in 2021, the highest in the European Union.[9] According to the Economist Intelligence Unit Worldwide Cost of Living Survey, in 2022, Paris was the city with the ninth-highest cost of living in the world"""},
    {"role": "assistant", "content": "BUDGET - 4, SAFE - 6, TOURISTIC - 10"},
    {"role": "user", "content":"CITY: Dublin is the capital city of Ireland.[11][12] On a bay at the mouth of the River Liffey, it is in the province of Leinster, bordered on the south by the Dublin Mountains, a part of the Wicklow Mountains range. At the 2022 census, the city council area had a population of 592,713, while Dublin and its suburbs had a population of 1,263,219, and County Dublin had a population of 1,501,500,[3][13][14] making it the largest city by population on the island of Ireland. A settlement was established in the area by the Gaels during or before the 7th century,[15] followed by the Vikings. As the Kingdom of Dublin grew, it became Ireland's principal settlement by the 12th century Anglo-Norman invasion of Ireland.[15] The city expanded rapidly from the 17th century and was briefly the second largest in the British Empire and sixth largest in Western Europe after the Acts of Union in 1800.[16] Following independence in 1922, Dublin became the capital of the Irish Free State, renamed Ireland in 1937. As of 2018, the city was listed by the Globalization and World Cities Research Network (GaWC) as a global city, with a ranking of Alpha minus, which placed it among the top thirty cities in the world.[17][18]"},
    {"role": "assistant", "content": "BUDGET - 3, SAFE - 8, TOURISTIC- 7"},
    {"role": "user", "content":"CITY: Marrakesh or Marrakech is the fourth-largest city in Morocco.[2] It is one of the four imperial cities of Morocco and is the capital of the Marrakesh–Safi region. The city lies west of the foothills of the Atlas Mountains. The city was founded circa 1070 by Abu Bakr ibn Umar as the capital of the Almoravid dynasty. The Almoravids established the first major structures in the city and shaped its layout for centuries to come. The red walls of the city, built by Ali ibn Yusuf in 1122–1123, and various buildings constructed in red sandstone afterwards, have given the city the nickname of the Red City or Ochre City. Marrakesh grew rapidly and established itself as a cultural, religious, and trading center for the Maghreb. After a period of decline, Marrakesh regained its status in the early 16th century as the capital of the Saadian dynasty, with sultans Abdallah al-Ghalib and Ahmad al-Mansur embellishing the city with an array of sumptuous monuments. Beginning in the 17th century, the city became popular among Sufi pilgrims for its seven patron saints who are buried here. In 1912, the French Protectorate in Morocco was established and T'hami El Glaoui became Pasha of Marrakesh and generally held this position until the independence of Morocco and the reestablishment of the monarchy in 1956."},
    {"role": "assistant", "content": "BUDGET - 8, SAFE - 2, TOURISTIC - 6"},
    {"role": "user", "content":"CITY: Perth is the capital city of Western Australia. It is the fourth most populous city in Australia, with a population of over 2.3 million within Greater Perth as of 2023. It is part of the South West Land Division of Western Australia, with most of Perth's metropolitan area on the Swan Coastal Plain between the Indian Ocean and the Darling Scarp. The city has expanded outward from the original British settlements on the Swan River, upon which its central business district and port of Fremantle are situated. Perth was founded by Captain James Stirling in 1829 as the administrative centre of the Swan River Colony. The city is situated on the traditional lands of the Whadjuk Noongar people, where Aboriginal Australians have lived for at least 45,000 years. Perth was named after the city of Perth in Scotland.[6] Perth was proclaimed as a city by Queen Victoria in 1856,[7] although the City of Perth currently governs only a small area around the central business district. Substantial population growth occurred during the late 19th-century Western Australian gold rushes, and the city has continued to expand, particularly after World War II due to a high net migration rate. Post-war immigrants were predominantly from the British Isles and Southern Europe, while more recent arrivals see a growing population of Asian descent. During the late 20th and early 21st centuries, a series of mining booms in various regions of Western Australia propelled Perth into the role of the regional headquarters for significant mining operations."},
    {"role": "assistant", "content": "BUDGET - 4, SAFE - 10, TOURISTIC - 5"},
]

In [21]:
response = client.chat.completions.create(
            model=model,
            messages = messages_with_few_shot + [{"role": "user", "content":city1}],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)


BUDGET - 6, SAFE - 4, TOURISTIC - 9


In [22]:
response = client.chat.completions.create(
            model=model,
            messages = messages_with_few_shot + [{"role": "user", "content":city2}],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET - 7, SAFE - 4, TOURISTIC - 6


In [23]:
response = client.chat.completions.create(
            model=model,
            messages = messages_with_few_shot + [{"role": "user", "content":city3}],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET - 3, SAFE - 9, TOURISTIC - 7


In [24]:
response = client.chat.completions.create(
            model=model,
            messages = messages_with_few_shot + [{"role": "user", "content":city4}],
            max_tokens=200,
            temperature=0.01
        )
print(response.choices[0].message.content)

BUDGET - 5, SAFE - 4, TOURISTIC - 8


In [37]:
# Bulk operations

def create_suggestion(career):
    response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content":"You are a career professional and tell people whether they would succeed or not in a career"},
                {"role": "user", "content": f"""I want to know with what to work with!

Consider the below details from my personality, between $$$ delimiters:

###BUSINESS DESCRIPTION###
$$$
{career}
$$$

Consider the below details from my personality, between $$$ delimiters:

###PERSONALITY###
$$$
I am an empathetic professional who likes to work in a team.
I am a very high risk taker and I usually take decisions and the responsibility.
I prefer to make wrong decisions over no decisions.
I like to learn new things and expand my knowledge, although some of them are not helpful for my career.
I like to do different things, but I don't like to not know at all how my routine is going to be the next day.

Key Traits:

Empathetic and compassionate
Curious and eager to learn
Detail-oriented with strong organizational skills
A collaborative team player
Logical but risk-taker


$$$

###TASK###
Give me an explanation about your response whether I would succeed or not.
The list should have a pro and a con with a maximum of 400 words each. You must do a good job, otherwise I will live with my mother forever."""},
            ],
            max_tokens=400,
            temperature=1
        )
    return response.choices[0].message.content

### NOTE: This is not me! I am doing it for fun ;)

In [38]:
career1 = "A lawyer is trained in the law and helps people solve legal problems. They provide advice, represent clients in court, and help with legal documents like contracts or wills. Lawyers can work for individuals, businesses, or the government to ensure that laws are followed and people’s rights are protected. Lawyers specialize in different areas of the law, such as criminal law, family law, or business law. For example, a criminal lawyer defends someone accused of a crime, while a family lawyer might help with divorces or child custody cases. No matter their specialty, lawyers use their knowledge of the law to find solutions, negotiate agreements, and advocate for their clients' best interests."
career2 = "Mary Kay consultants, called beauty consultants, can make income by directly selling to people in their community, and also receive a commission when they recruit others to begin selling under their distribution network."
career3 = "A broker is a person or entity that arranges transactions between a buyer and a seller. This may be done for a commission when the deal is executed. A broker who also acts as a seller or as a buyer becomes a principal party to the deal."
career4 = "An Ornithologist study birds in their natural habitats or in the laboratory. They also may write research reports and proposals for grants, teach classes, present research to the public, and have administrative duties related to these activities."

In [39]:
career_suggestion = create_suggestion(career1)
print(career_suggestion)

Based on your personality traits and the business description of a lawyer, here’s an analysis of whether you would succeed in a legal career, including both pros and cons.

**Pros:**
1. **Empathy and Compassion**: Your empathetic nature is an asset in law. Lawyers often need to understand their clients' feelings and perspectives to advocate effectively and provide the necessary support during difficult legal matters. This skill can help forge strong client relationships, making you a trusted advisor in sensitive cases like family law or criminal defense.
   
2. **Collaborative Team Player**: Law often involves working with a team, whether it’s collaborating with colleagues or consulting experts. Your preference for teamwork aligns well with the legal environment, where collaboration can lead to better outcomes for clients and a more harmonious workplace.

3. **Curiosity and Eagerness to Learn**: Your desire to continuously learn is crucial in law, a field that evolves with legislation 

In [40]:
career_suggestion2 = create_suggestion(career2)
print(career_suggestion2)

Considering your personality traits and how they align with a career as a Mary Kay beauty consultant, here's an analysis of your potential success in this field:

### Pros

1. **Empathetic and Compassionate**: Your empathetic nature is a strong asset in sales, especially in the beauty industry where understanding customer needs and building relationships are crucial. Clients are more likely to trust and buy from someone who genuinely cares about their preferences and concerns.

2. **Team Player**: The collaborative aspect of your personality will serve you well when recruiting others to your network. Working effectively with a team can help you boost morale and achieve shared goals, which is essential in a consultancy model that thrives on group dynamics.

3. **Risk-Taking**: Your inclination toward risk-taking is beneficial in a direct sales environment where new product launches and dynamic marketing strategies are common. Being willing to try innovative approaches can help different

In [41]:
career_suggestion3 = create_suggestion(career3)
print(career_suggestion3)

Based on your personality traits and the career description of a broker, here’s a breakdown of the pros and cons to determine your potential success in this field.

### Pros:
1. **Empathy and Compassion**: Your empathetic nature would allow you to connect with clients, understand their needs, and establish trust. This trait is crucial in a brokerage role, as it can lead to repeat business and long-term relationships.

2. **Collaborative Team Player**: Brokers often work in teams, whether in a firm or with clients and other stakeholders. Your preference for teamwork can enhance your ability to collaborate effectively, leading to better outcomes for clients and a supportive work environment.

3. **Willingness to Learn**: Your eagerness to learn new things can be a significant asset in the constantly changing financial markets. This adaptability allows you to stay on top of market trends, regulations, and new strategies, which is crucial for success as a broker.

4. **Decisiveness Under R

In [42]:
career_suggestion4 = create_suggestion(career4)
print(career_suggestion4)

Based on your personality traits and the nature of the career as an ornithologist, let’s explore the potential for success in this field.

### Pros

1. **Empathy and Communication Skills**: Your empathetic nature will serve you well in collaborative environments, such as research teams. Being able to communicate effectively and understand the perspectives of others can foster a productive work atmosphere and enhance your ability to present complex information to diverse audiences.

2. **Curiosity and Eagerness to Learn**: Ornithology requires a continual engagement with new research, methods, and findings. Your intrinsic motivation to learn will drive you to stay abreast of developments in the field, participate in relevant training sessions, and possibly pursue further education, benefiting both your career and the broader scientific community.

3. **Detail Orientation**: Your attention to detail is a critical asset in the scientific realm. Conducting research, writing papers, and col


## Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong <br>

## What did you learn? <br>


There are clear differences between the model 4o mini and 3-turbo. In the mini model, the classifications tend to be more volatile, and not always the text has the format desired.

However, these models do a very good work, with reasonable responses. The prompts are very usefull to organise the output. In terms of format, the bulk operations seem to be more reliable, when comparing with the zero or one shot.

The classification with the shots are very good. Although the temperature is 0.01, the new scores seem to reflect better the reality of the places, based on wikipedia descriptions. It is definitely a great way to run classifications on a business.


